In [1]:
import sys
from pathlib import Path

BASEDIR = Path("/workspaces/HARP/") / "src"  # Replace with your own basedir path for the repo

sys.path.insert(0, str(BASEDIR))

In [2]:
from __future__ import annotations

import copy
import itertools
import json
import math
import os
import pickle
import pprint
import re
import textwrap
import time
import traceback
from collections import Counter, defaultdict
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, clear_output, display
from tqdm.auto import tqdm

from openai import OpenAI

In [3]:
from eval.eval import create_batch
from eval.prompts import *
from eval.utils import read_jsonl, write_jsonl, get_uid

# Data

In [4]:
dataset = read_jsonl(BASEDIR / "data/processed/HARP.jsonl")
dataset_map = {get_uid(p): p for p in dataset}
len(dataset)

4780

# Run eval

In [12]:
client = OpenAI()

## Create batch

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

In [13]:
gpt4_system_instruction

'You are a helpful assistant.'

In [14]:
modified_openai_o1_prompt_template

'Solve the following math problem. The last line of your response should be of the form "Answer:\n$ANSWER" (without quotes) where $ANSWER is the answer to the problem. Remember to put your answer on its own line after "Answer:", and you do not need to use a \\boxed command.\n\n    {problem}'

In [15]:
batch = create_batch(
    dataset,
    api="openai",
    model="gpt-4o-mini-2024-07-18",
    fewshot_messages=[],
    system_prompt=gpt4_system_instruction,
    user_prompt_template=modified_openai_o1_prompt_template,
    ending_assistant_prompt=None,
    max_tokens=2048,
    temperature=0,
    seed=0,
    logprobs=True,
    top_logprobs=None,
    # just to remove irrelevant params
    top_p=None,
)

In [16]:
write_jsonl(batch, BASEDIR / "inputs/short_answer/gpt-4o-mini-2024-07-18/batch.jsonl")

In [17]:
with open(BASEDIR / "inputs/short_answer/gpt-4o-mini-2024-07-18/batch.jsonl", "rb") as f:
    batch_input_file = client.files.create(file=f, purpose="batch")

In [18]:
batch_input_file.id

'file-JPFPVMLebKTzoFhTzCkpEZKn'

## Run batch

In [19]:
client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "gpt-4o-mini-2024-07-18 eval no-top-logprobs"
    }
)

Batch(id='batch_672d48e31fcc8190bd8d281c756d2c67', completion_window='24h', created_at=1731021027, endpoint='/v1/chat/completions', input_file_id='file-JPFPVMLebKTzoFhTzCkpEZKn', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731107427, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18 eval no-top-logprobs'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
BATCH_NAME = "batch_672d48e31fcc8190bd8d281c756d2c67"  # Replace with your own

## Check on status

In [21]:
client.batches.retrieve(BATCH_NAME)

Batch(id='batch_672d48e31fcc8190bd8d281c756d2c67', completion_window='24h', created_at=1731021027, endpoint='/v1/chat/completions', input_file_id='file-JPFPVMLebKTzoFhTzCkpEZKn', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731107427, failed_at=None, finalizing_at=None, in_progress_at=1731021028, metadata={'description': 'gpt-4o-mini-2024-07-18 eval no-top-logprobs'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=4794))

## Retrieve

In [22]:
outfile = client.batches.retrieve(BATCH_NAME).output_file_id
results = client.files.content(outfile)

In [23]:
results.write_to_file(BASEDIR / "outputs/short_answer/gpt-4o-mini-2024-07-18/outputs.jsonl")